# Extraction des différents sites statiques (Emploi territorial & JobsthatmakeSense)

In [ ]:
from bs4 import BeautifulSoup

from datetime import datetime
import pandas as pd
import random
import re
import requests
import time 


# Emploi territorial

In [ ]:
URL = "https://www.emploi-territorial.fr/emploi-mobilite/?adv-search=data&search-fam-metier=A7" # Nous récupérons simplement la première page (unique qui renvoie des offres véritablement en relation avec la data)

# On crée un dictionnaire pour récupérer les informations pertinentes.
dict_emploiterr = {
    "entreprise": [],
    "titre": [],
    "type_contrat": [],
    "ville": [],
    "departement": [],
    "pays": [],
    "link": [],
    "date_publication": [],
    "descriptif": [],
    "profil": []
}


On intéragit tout d'abord avec la page principale de recherche.

In [ ]:
try:
    page = requests.get(URL)
    # Vérifie que la requête a été correctement réalisée
    page.raise_for_status()  

    # Crée un objet BeautifulSoup représentant le document HTML
    soup = BeautifulSoup(page.content, "html.parser")
    # On extrait la partie de la page qui contient les offres
    results = soup.find_all("tr", {"id": True})


    for job in results:
        # On extrait le titre de l'offre d'emploi et on nettoie le texte
        title = job.find("div", class_="detail-offre detail-offre-titre mb-1").getText().replace("\n","").replace("(h/f)","").replace("(H/F)", "").strip()
        dict_emploiterr["titre"].append(title)
        # On extrait le nom de la collectivité
        entreprise = job.find("span", class_="valeur font-weight-bold")
        if entreprise:
            entreprise = entreprise.getText().replace("\n","").strip()
        else:
            entreprise = "Non spécifié"
        dict_emploiterr["entreprise"].append(entreprise)
        # On extrait le département de la collectivité qui recrute
        departement = job.find("span", class_="font-weight-bold text-secondary set-font-size-0-9em ml-2")
        if departement:
            departement = departement.getText().replace("\n","").strip()
        else:
            departement = "Non spécifié"
        dict_emploiterr["departement"].append(departement)

        # On extrait le lien de l'offre d'emploi et on lui ajoute le préfixe pour être complet
        link = job.find("a", href=True)["href"]
        dict_emploiterr["link"].append("https://www.emploi-territorial.fr" + link)

        # On extrait la date de publication
        publication = job.find("button", class_="btn btn-icon col text-center")["data-tooltip"].split("publié le ")[1]
        date = datetime.strptime(publication, '%d/%m/%Y')
        dict_emploiterr["date_publication"].append(date)

        dict_emploiterr["type_contrat"].append("Fonction Publique")

except requests.RequestException as e:
    print(f"An error occurred: {e}")

In [ ]:
dict_emploiterr

Avant d'aller directement sur chaque offre.

In [ ]:
# Pour chaque lien de chaque offre,
for link in dict_emploiterr["link"]:
    try:
        print(link)
        # On accède à l'URL de l'offre d'emploi
        job_page = requests.get(link)
        # On vérifie la réussite de la connection
        job_page.raise_for_status()

        # On transforme cela en objet BeautifulSoup
        soup2 = BeautifulSoup(job_page.content, "html.parser")
        
        # On crée une liste vide de paragraphes
        liste_paragraphe = []
        # On extrait les différentes parties contenant le texte de l'offre
        texte = soup2.find_all("div", class_="offre-item-text row px-3")
        # Pour chaque carré de texte,
        for paragraphes in texte: 
            # On récupère le texte et on le nettoie
            par_clean = paragraphes.getText().replace("\n", " ").replace("\t", " ").replace("\'", "'").strip()
            par_spaceout = re.sub(r'\s+'," ",par_clean)
            # On l'ajoute à une liste de paragraphes
            liste_paragraphe.append(par_spaceout)
        # On ajoute les différentes paragraphes aux différentes listes du dictionnaire
        dict_emploiterr["descriptif"].append(liste_paragraphe[0] + liste_paragraphe[1])
        dict_emploiterr["profil"].append(liste_paragraphe[2])
        
        # On extrait la ville et le pays de l'entreprise
        adresse = soup2.find("div", class_="offre-item-value col-8").getText().split()
        ville = adresse[len(adresse)-1]
        dict_emploiterr["ville"].append(ville.capitalize())
        dict_emploiterr["pays"].append("France")


        # On applique un délai aléatoire de 1 à 5 secondes entre chaque page pour ne pas surcharger les serveurs
        time.sleep(random.uniform(1, 5))

    except requests.RequestException as e:
        print(f"An error occurred: {e}")

In [ ]:
df_emploiterr = pd.DataFrame(dict_emploiterr)
df_emploiterr["salaire"] = "Non spécifié"
df_emploiterr["experience"] = "Non spécifié"
df_emploiterr["source"] = 'Emploi-Territorial'
df_emploiterr

In [ ]:
# On extrait le fichier csv des offres récupérées.
df_emploiterr.to_csv("empterritorial_10122023.csv", index=None, encoding="utf-8-sig")

# Jobs that make sense

In [ ]:
# Dictionnaire pour récupérer les informations pertinentes
dict_makesense = {
    "entreprise": [],
    "titre": [],
    "type_contrat": [],
    "ville": [],
    "pays": [],
    "link": [],
    "date_publication": [],
    "descriptif": [],
    "profil": [], 
    "salaire" : [],
    "experience": [],
    "source": []
}

In [ ]:
# On crée une liste des différentes pages
URL_list = []

# On crée l'URL pour chaque page
for n in range(0,5): # On commence à partir de zéro pour respecter le format du site de JobsMakeSense
    if n == 0 :
        URL_list.append("https://jobs.makesense.org/fr/s/jobs/all?s=data&sortBy=relevance")
    URL_list.append(f'https://jobs.makesense.org/fr/s/jobs/all?s=data&sortBy=relevance&items_page={n}')

On intéragit tout d'abord avec la page principale de recherche. Voici un exemple avec juste la page 1 (recherche du 11/01/2024).

In [ ]:
for pages in URL_list: 
    try:
        print(pages)
        page = requests.get(pages)
        # Vérifie que la requête a été correctement réalisée
        page.raise_for_status()  

        # Crée un objet BeautifulSoup représentant le document HTML
        soup = BeautifulSoup(page.content, "html.parser")
        # On extrait la partie de la page qui contient les offres
        results = soup.find(class_="componentList")

        # On récupère tous les éléments contenant "aria-label", càd chaque offre.
        job_liste = results.find_all("div", {"aria-label": True})
        # Pour chaque "carte" d'offre d'emploisur la page de requête
        for job in job_liste:
            # On extrait le lien de l'offre d'emploi et on lui ajoute le préfixe pour être complet
            link = job.find("a", href=True)["href"]
            if "programs" not in link:
                dict_makesense["link"].append("https://jobs.makesense.org" + link)
                # On extrait le titre de l'offre d'emploi et on nettoie le texte
                title = job.find("h3", class_="content__title").getText().replace("\n","").strip()
                dict_makesense["titre"].append(title)
                
                # On extrait les méta-informations de la carte
                details = job.find("div", class_="job__meta job__metas").getText()

                # On nettoie la chaine de caractère
                details = details.replace("💡", "")
                # Les mots sont séparés par des séries d'espaces, on sépare les différents items
                details = re.split(r'\s{2,}', details)
                # Sur ce site, le nom de l'entreprise occupe la première place
                if details[1]:
                    dict_makesense["entreprise"].append(details[1])
                # S'il est noté, le type de contrat occupe la troisième place
                if len(details) > 2:
                    dict_makesense["type_contrat"].append(details[3])

        # On applique un délai aléatoire de 1 à 3 secondes entre chaque page pour ne pas surcharger les serveurs
        time.sleep(random.uniform(1, 3)) 

    except requests.RequestException as e:
        print(f"An error occurred: {e}")



In [ ]:
dict_makesense


In [ ]:
dict_makesense["link"]

In [ ]:
len(dict_makesense["entreprise"])

In [ ]:
dict_makesense

In [ ]:
# Pour chaque lien de chaque offre,
for link in dict_makesense["link"]:
    if "program/" not in link:
        try:
            print(link)
            # On accède à l'URL de l'offre d'emploi
            job_page = requests.get(link)
            # On vérifie la réussite de la connection
            job_page.raise_for_status()

            # On transforme cela en objet BeautifulSoup
            soup2 = BeautifulSoup(job_page.content, "html.parser")
            
            # On extrait la description de l'offre
            dict_makesense["descriptif"].append(soup2.find("main", class_="job__main-content").getText().replace("\n", " "))
            # On extrait le profil recherché
            try:
                dict_makesense["profil"].append(soup2.find("div", class_="job__main-content").getText().replace("\n", " "))
            except: 
                dict_makesense["profil"].append("Non spécifié")
            # On extrait la ville et le pays de l'entreprise
            lieu = soup2.find("address").getText().replace("\n", " ").strip()
            if "," in lieu:
                split_lieu = lieu.split(", ")
                if len(split_lieu) == 2:
                    ville, pays = split_lieu
                    dict_makesense["ville"].append(ville)
                    dict_makesense["pays"].append(pays)
                else:
                    dict_makesense["ville"].append(lieu)
                    dict_makesense["pays"].append("Non Spécifié")                   
            else: 
                dict_makesense["ville"].append(lieu)
                dict_makesense["pays"].append("Non Spécifié")

            # On extrait la date de publication et on ne garde que la date en format dd/mm/yyyy
            if "Publié" in soup2.get_text():
                publication = soup2.find(text=lambda text: "Publiée" in text).replace("\n", '').strip().split("Publiée le ")[1]
                date = datetime.strptime(publication, '%d/%m/%Y')
                dict_makesense["date_publication"].append(date)
            else: 
                dict_makesense["date_publication"].append("Non spécifié")
            dict_makesense["salaire"].append("Non spécifié")
            dict_makesense["experience"].append("Non spécifié")
            dict_makesense["source"].append("JobsThatMakeSense")

            # On applique un délai aléatoire de 1 à 10 secondes entre chaque page pour ne pas surcharger les serveurs
            time.sleep(random.uniform(1, 10))

        except requests.RequestException as e:
            print(f"An error occurred: {e}")


In [ ]:
pd.DataFrame(dict_makesense)

In [ ]:
pd.DataFrame(dict_makesense).to_csv("makesense_11012024.csv", index=None, encoding="utf-8-sig")